In [4]:
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Inference and learning final project

## Introduction


## Creating Neural Networks

In [3]:
class D_Conv(torch.nn.Module):
    def __init__(self, base_channel_size, local = False, FC = False) -> None:
        alpha = base_channel_size
        super().__init__()
        modules = []
        size_conv = [
            (3,0),
            (alpha,1),
            (2*alpha,2),
            (2*alpha,1),
            (4*alpha,2),
            (4*alpha,1),
            (8*alpha,2),
            (8*alpha,1),
            (16*alpha,2),
            (64*alpha,2)]
        for i in range(1,len(size_conv)):
            modules.extend([
                torch.nn.Conv2d(
                    in_channels= size_conv[i-1][0],
                    out_channels=size_conv[i][0],
                    kernel_size=3,
                    stride=size_conv[i][1]),
                torch.nn.BatchNorm2d(size_conv[i][0]),
                torch.nn.ReLU()
            ])
        self.conv = torch.nn.Sequential(*modules)
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(16*alpha*4*4, 64*alpha),
            torch.nn.BatchNorm1d(64*alpha),
            torch.nn.ReLU(),
            torch.nn.Linear(64*alpha, 10),
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

class S_Conv(torch.nn.Module):
    def __init__(self, base_channel_size, local = False, FC = False) -> None:
        alpha = base_channel_size
        super().__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(
                in_channels=3,
                out_channels= alpha,
                kernel_size=9,
                stride = 2),
            torch.nn.BatchNorm2d(alpha),
            torch.nn.ReLU()
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(alpha*4*4, 24*alpha),
            torch.nn.BatchNorm1d(24*alpha),
            torch.nn.ReLU(),
            torch.nn.Linear(24*alpha, 10),
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    

device(type='cuda')